# Basic Prototype 1

This prototype will just test the basic functionality of the broadcast and live youtube instruction loop. Very minimal prototype. Live is initiated manually using OBS, the live chat is then scraped using pytchat, the live chat (numbers 1-5) is then used to pick which video to play using opencv. The video is then broadcast live to youtube via OBS as a closed loop with a slight delay. 

There is code to try to automate the broadcast using OBS websocket, but authentication is required manually anyways, so it's just easier to start the broadcast manually

In [1]:
# get live chat

import pytchat
# test lofi girl live
# https://www.youtube.com/watch?v=jfKfPfyJRdk

# test moonrabbit: https://www.youtube.com/watch?v=op_CtC7mJms

chat = pytchat.create(video_id="jfKfPfyJRdk")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")

2023-08-16 18:18:58 [Ayush]- :moai:
2023-08-16 18:18:59 [kim lilly 😍]- devil 10th pachi ni life barbadd na dost na pyaar:rolling_on_the_floor_laughing::rolling_on_the_floor_laughing:
2023-08-16 18:19:00 [Moinnnnnnn]- #Jasmine , ihy
2023-08-16 18:19:01 [Devil_]- #kim lilly naresh chomu chhe :face_with_tears_of_joy: ana jode vat na kar :rolling_on_the_floor_laughing:


In [3]:
import glob

# testing playing video with choice - choices are numbers 1-5

def make_videos_dict():
    # files = glob.glob('/Volumes/My Passport/UAL_Thesis/raw_videos/*')
    files = glob.glob('G:/UAL_Thesis/raw_videos/*')
    files = [f.split('/')[-1] for f in files]
    files_dict = {str(i+1): value for i, value in enumerate(files)}

    return files_dict
  
def test_choose_videos(i , videos_dict):
    return videos_dict[i]

import cv2
import time

# play video according to live chat command using cv2 
# raw_videos_path = '/Volumes/My Passport/UAL_Thesis/raw_videos'
raw_videos_path = 'G:/UAL_Thesis/raw_videos/*'
videos_dict = make_videos_dict()

def play_video(video_path, play_time=5):
    cap = cv2.VideoCapture(video_path)

    start_time = time.time()
    while(cap.isOpened() and time.time() - start_time < play_time):
        ret, frame = cap.read()
        if ret == True:
            cv2.imshow('Frame', frame)
            # Press Q on keyboard to exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows


# while True:
#     video_file_input = input("Enter the video file path choice to play or 'q' to quit: ")
#     if video_file_input.lower() == 'q':
#         break
#     else:
#         video_file = raw_videos_path + '/'+test_choose_videos(video_file_input, videos_dict)
#         print(video_file)
#         play_video(video_file)


In [6]:
# choose videos using live chat

# video_id = 
import pytchat
# windows
raw_videos_path = 'G:/UAL_Thesis' 

# live stream must start first, then copy the id from the live stream URL
chat = pytchat.create(video_id="zKsdVr4yo2s")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        video_file = raw_videos_path + '/'+test_choose_videos(c.message, videos_dict)
        print(video_file)
        play_video(video_file)

2023-08-16 18:43:45 [Moon Rabbit]- 1
G:/UAL_Thesis/raw_videos\idle_bob_1.MOV
2023-08-16 18:45:37 [Moon Rabbit]- 3
G:/UAL_Thesis/raw_videos\jump_out_in.MOV


: 

In [5]:
import configparser

# Load the ini file
config = configparser.ConfigParser()
config.read('auth.ini')

# Access the values using the section and key
port = int(config['OBS Credentials']['port'])
host = config['OBS Credentials']['host']
pwd = config['OBS Credentials']['pwd']




In [ ]:
# test obs websocket - it works but, it's not needed really, it's easier and more robust to just start the stream manually

import cv2
import pytchat
from obswebsocket import obsws, requests
import pygetwindow as gw
import pyautogui
import time

# Connect to OBS - requires AUTH file

ws = obsws(host, port,pwd)
ws.connect()



# don't forget to manually select the live stream in OBS first before starting to stream
# also stream will not be present if previous livestream edit window is not closed on youtube studios

    # Start streaming
ws.call(requests.StartStreaming())





In [7]:
# prototype-1
# Have to run it twice, once to start the OBS stream, then copy the video id from the started stream to scrape the chat, then run again for it to work.
# As said, it works, but still requires manual authorisation in OBS, so might as well just start the stream manually and save maintaining all this OBS code.
# live stream url: https://youtu.be/nPdEhg5zNMU

import cv2
import pytchat
from obswebsocket import obsws, requests
import pygetwindow as gw
import time

# Connect to OBS

ws = obsws(host,port,pwd)
ws.connect()

# List all profiles (broadcasts)
# current_settings = ws.call(requests.GetStreamSettings())
# print(current_settings)

# create scene
scene_request = requests.CreateScene()
scene_request.scene_name = "OpenCV_Scene"
scene_name =  "OpenCV_Scene"
ws.call(scene_request)


# Add a "Window Capture" source to the scene
source_name = "OpenCV_Window_Capture"
window_name = "Frame"  # Name of the OpenCV window (as per cv2.imshow('Frame', frame))
source_request = requests.CreateSource()
source_request.sourceName = source_name
source_request.sourceType = "window_capture"
source_request.sourceSettings = {
    "window": window_name + ":.*",  # This regex ensures it matches the window title even if there are additional characters
    "window_match_priority": 1  # Match by window title
}
ws.call(source_request)

# Add the created source to the scene
ws.call(requests.AddSceneItem(scene_name, source_name, -1))




# don't forget to manually select the live stream in OBS first before starting to stream
# also stream will not be present if previous livestream edit window is not closed on youtube studios

    # Start streaming
ws.call(requests.StartStreaming())



chat = pytchat.create(video_id="nPdEhg5zNMU")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        video_file = raw_videos_path + '/'+test_choose_videos(c.message, videos_dict)
        print(video_file)
        play_video(video_file)

ws.disconnect()

WebSocketAddressException: [Errno 11001] getaddrinfo failed